Import Libraries

In [1]:
import os
import sys
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from time import time
import matplotlib.pyplot as plt
from skimage.color import gray2rgb
from scipy.misc import imread, imsave, imresize, imshow

from keras.utils import np_utils
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

%matplotlib inline

%load_ext autoreload
%autoreload 2

from keras.models import model_from_json

from ernest import *

Using Theano backend.


Load training data
Split the training data into train and validation set

In [2]:
X_train, y_train, X_val, y_val, class_labels = process_train('train', 3, 224, 224, 0.1)

Number of classes in training: 16
Total no. of images in training: 4591
Processing class: 0 , Automobile
Processing class: 1 , Comics_and_Cartoons
Processing class: 2 , Famous_Personalities
Processing class: 3 , Festivals_and_Occasions
Processing class: 4 , Graffiti_and_Illustrations
Processing class: 5 , Movies_and_TV_shows
Processing class: 6 , Music
Processing class: 7 , Nature
Processing class: 8 , No_Theme
Processing class: 9 , Patterns_and_Ethnic
Processing class: 10 , Signs_and_Symbols
Processing class: 11 , Spiritual
Processing class: 12 , Sports
Processing class: 13 , Superheroes
Processing class: 14 , Typography
Processing class: 15 , Vintage
Time taken: 277.882999897
Completed Processing
Size of training set: (4132L, 3L, 224L, 224L)
Size of training labels: (4132L, 16L)
Size of validation set: (459L, 3L, 224L, 224L)
Size of validation labels: (459L, 16L)


Define a model with VGG16 architecture

In [3]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_2[0][0]            
___________________________________________________________________________________________

Train only the last 3 fully connected layers of the model
Define all other layers as 'non-trainable'

In [5]:
for i in xrange(len(model.layers)-5):
    model.layers[i].trainable = False

Load pre-trained weights

Source for pre-trained weights: https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3

In [6]:
model.load_weights('vgg16_weights.h5')

Define a function to remove the last layer of the model

In [7]:
def pop_layer(model):
    if not model.outputs:
        raise Exception('Sequential model cannot be popped: model is empty.')
    
    model.layers.pop()
    if not model.layers:
        model.outputs = []
        model.inbound_nodes = []
        model.outbound_nodes = []
    else:
        model.layers[-1].outbound_nodes = []
        model.outputs = [model.layers[-1].output]
    model.built = False

Remove the last layer of the model

In [8]:
pop_layer(model)

Add new layer whose output equals the number of classes in the dataset

Define SGD and compile the model

In [9]:
model.add(Dense(16, activation='softmax'))
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy', 'fmeasure'])

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_2[0][0]            
___________________________________________________________________________________________

Train the model

In [11]:
model.fit(X_train, y_train, nb_epoch=5, batch_size=128, validation_data = (X_val, y_val), verbose=1)

Train on 4132 samples, validate on 459 samples
Epoch 1/5
4132/4132 [==============================] - 5597s - loss: 2.2760 - acc: 0.4530 - fmeasure: 0.4300 - val_loss: 1.3174 - val_acc: 0.6253 - val_fmeasure: 0.5820

Save model and model weights

In [13]:
json_model = model.to_json()
with open("0412-model.json", "w") as json_file:
    json_file.write(json_model)

In [14]:
model.save_weights('0412-model_weights_1.h5')

Predict on test set

In [22]:
subm = predict_test('Test', model, 224, 224, class_labels)

Processing test image 0 of 1200
Processing test image 100 of 1200
Processing test image 200 of 1200
Processing test image 300 of 1200
Processing test image 400 of 1200
Processing test image 500 of 1200
Processing test image 600 of 1200
Processing test image 700 of 1200
Processing test image 800 of 1200
Processing test image 900 of 1200
Processing test image 1000 of 1200
Processing test image 1100 of 1200


Create submission file

In [23]:
subm.to_csv('subm_0412.csv', index = False)